<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #2: Adquisición de textos
`Fecha de entrega: Agosto 29, 2020. (Antes del inicio de la próxima clase).`

# Punto 1:

- Descomprimir el archivo `.zip` de los poemas
- Leer cada uno de sus archivos
- Responder: ¿Cuál archivo tiene el mayor número de palabras?

# Punto 2:

- Hacer Web Scraping de 10 biografías en Wikipedia (en búcle)
- Obtener el **encabezado** de cada biografía
- Obtener todos los **contenidos** y **etiquetas de título** asociados a los **links** del primer párrafo

# Solución

## 🛑 Punto 1

In [1]:
import os
import shutil
import operator
import urllib.request
import bs4 as bs
from zipfile import ZipFile

### 🤖 Funcion para descomprimir .zip

In [2]:
def unzip_file(zip_file, target_folder):
    """ Descomprime el cotenido de [zip_file] en la carpeta [target_folder]
        Devuelve la ruta final del contenido. """
    try:
        with ZipFile(zip_file) as archivo: 
            archivo.extractall(target_folder)
            shutil.rmtree(target_folder+'/__MACOSX')
        return target_folder + '/' + zip_file.replace('.zip','/') 
    except Exception as e:
        print('Error descomprimiendo el archivo [{}] Mensaje: {}'.format(zip_file, str(e)))
        return None

### 🤖 Función para obtener los nombres de archivos presentes en una ruta dada

In [3]:
def get_file_names_in_path(path):
    """ Devuelve una lista con los nombres de archivos en la ruta [path] """
    filenames = []
    try:
        for raiz, dirs, archivos in os.walk(path):
            for archivo in archivos:
                filenames.append(path+archivo)
        return filenames
    except Exception as e:
        print('Error leyendo los archivo en [{}] Mensaje: {}'.format(path, str(e)))
    return None

### 🤖 Función para leer archivo y contar palabras en cada uno

In [4]:
def read_file_and_count_words(file_list):
    """ Devuelve una lista ordeneda de tuplas con el nombre del archivo y la cantidad de palabras que contiene. """
    word_counter = {}
    for file in file_list:
        with open(file) as archivo: 
            data = archivo.read()
            word_counter[file] = len(data.split())
    sorted_word_counter = sorted(word_counter.items(), key=operator.itemgetter(1), reverse=True)
    return list(sorted_word_counter)

### ⚙️ Proceso

### 🤔 ¿Cuál archivo tiene el mayor número de palabras?

In [5]:
unzip = unzip_file('poemas.zip', 'unzip')

if unzip:
    files = get_file_names_in_path(unzip)
    if files:
        result = read_file_and_count_words(files)
        print("RTA: El archivo que más contiene palabras es [{}] con [{}] palabras.\n\n".format(result[0][0], result[0][1]))

RTA: El archivo que más contiene palabras es [unzip/poemas/Si╠ündrome (Mario Benedetti).txt] con [53] palabras.




## 🛑 Punto 2

### 🤖 Función para obtener el html en soup

In [6]:
def get_soup(link):
    """ Hacer petición GET a una URL dada y devolver el html como soup """
    response = urllib.request.urlopen(link)
    source = response.read() 
    response.close()
    soup = bs.BeautifulSoup(source,'html.parser')
    return soup

### 🤖 Función que obtiene los elementos h1, p, a, title

In [7]:
def call_wiki(link):
    """ Obtiene e imprime los datos de interes que se encuentran en la eqtiquetas. """
    soup = get_soup(link)

    print("="*100)
    print("• \033[1mUrl:\033[0m\n\t {}".format(link))
    print('• \033[1mEncabezado:\033[0m \n\t {}'.format(soup.find('h1', attrs={'id':'firstHeading'}).text))

    tags_a = soup.find('p').find_all('a')
    if len(tags_a) == 0:
        tags_a = soup.find_all('p')[1].find_all('a')
    
    if len(tags_a) > 0:
        print('• \033[1mLinks 1er parrafo:\033[0m')
        for tag_a in tags_a:
            print('\tContenido => {}'.format(tag_a.text))
            print('\tTitulo => {} \n'.format(tag_a.get('title')))
    else:
        print('El primer párrafo no contiene links.')

### 🤖 Función que extrae y construye los links para hacer scraping desde otra web

In [8]:
def build_link_to_scraping(main_link, base_url_wiki):
    """ Extrae nombres de personas desde [main_link] y genera urls con base en la url [base_url_wiki] """
    soup_best_programmers = get_soup(main_link)
    li_programmer_names = soup_best_programmers.find(class_="list").find_all("li")
    links_to_scraping = []

    for programmer_name in li_programmer_names:
        full_name = programmer_name.text.split()
        links_to_scraping.append(base_url_wiki + full_name[0]+'_'+full_name[1])
        
    return links_to_scraping

### ⚙️ Proceso

👉 `En main_link se encuentra un listado con los nombres de los mejores programadores a los que quiero hacerle scraping (para no quemar los nombres ni los links)`

In [9]:
# Defino URLs base
main_link = "https://www.tutorialspoint.com/who-are-currently-the-best-programmers-in-the-world"
base_url_wiki = "https://es.wikipedia.org/wiki/"
print(main_link)

https://www.tutorialspoint.com/who-are-currently-the-best-programmers-in-the-world


In [10]:
# Genero las demas URLs
links_to_scraping = build_link_to_scraping(main_link, base_url_wiki)

# Hacer GET de cada URL con reintento
for link in links_to_scraping:
    try:
        call_wiki(link)
    except Exception as e:
        print("Error con la url: [{}]. Mensaje: {}. Inicia reintento en idioma Inglés.".format(link,e))
        call_wiki(link.replace("es.","en."))
        pass

• Url:
	 https://es.wikipedia.org/wiki/Bill_Gates
• Encabezado: 
	 Bill Gates
• Links 1er parrafo:
	Contenido => Seattle
	Titulo => Seattle 

	Contenido => Washington
	Titulo => Washington (estado) 

	Contenido => [1]
	Titulo => None 

	Contenido => magnate empresarial
	Titulo => Magnate 

	Contenido => informático
	Titulo => Informático 

	Contenido => filántropo
	Titulo => Filantropía 

	Contenido => [3]
	Titulo => None 

	Contenido => estadounidense
	Titulo => Nacionalidad estadounidense 

	Contenido => Microsoft
	Titulo => Microsoft 

	Contenido => Paul Allen
	Titulo => Paul Allen 

	Contenido => [4]
	Titulo => None 

• Url:
	 https://es.wikipedia.org/wiki/James_Gosling
• Encabezado: 
	 James Gosling
• Links 1er parrafo:
	Contenido => OC
	Titulo => Orden de Canadá 

	Contenido => 19 de mayo
	Titulo => 19 de mayo 

	Contenido => 1955
	Titulo => 1955 

	Contenido => Calgary
	Titulo => Calgary, Alberta 

	Contenido => Alberta
	Titulo => Alberta 

	Contenido => Canadá
	Titulo => Canadá

• Url:
	 https://es.wikipedia.org/wiki/Mark_Zuckerberg
• Encabezado: 
	 Mark Zuckerberg
• Links 1er parrafo:
	Contenido => White Plains
	Titulo => White Plains (Nueva York) 

	Contenido => Estados Unidos
	Titulo => Estados Unidos 

	Contenido => 14 de mayo
	Titulo => 14 de mayo 

	Contenido => 1984
	Titulo => 1984 

	Contenido => programador
	Titulo => Programador 

	Contenido => empresario
	Titulo => Empresario 

	Contenido => estadounidense
	Titulo => Nacionalidad estadounidense 

	Contenido => Facebook
	Titulo => Facebook 

	Contenido => Universidad de Harvard
	Titulo => Universidad de Harvard 

	Contenido => ciencias de la computación
	Titulo => Ciencias de la computación 

	Contenido => Eduardo Saverin
	Titulo => Eduardo Saverin 

	Contenido => Dustin Moskovitz
	Titulo => Dustin Moskovitz 

	Contenido => Chris Hughes
	Titulo => Chris Hughes 

	Contenido => 2018
	Titulo => 2018 

	Contenido => lista de multimillonarios
	Titulo => Anexo:Multimillonarios según Forbes 

	Contenido => 